In [40]:
from dotenv import load_dotenv
load_dotenv()
cohere_api_key=os.getenv("COHERE_API_KEY")
alpha_vantage_api_key=os.getenv("ALPHA_VANTAGE_API_KEY")

In [49]:
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.llms import Cohere
import requests

# Function to fetch financial data from Alpha Vantage API
def fetch_financial_data(api_key, ticker):
    """
    Fetch financial data for the given ticker using Alpha Vantage API.
    Returns a dictionary with revenue, net income, liabilities, and assets.
    """
    base_url = "https://www.alphavantage.co/query"
    income_params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key
    }
    balance_params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": api_key
    }

    # Fetch Income Statement
    income_response = requests.get(base_url, params=income_params)
    if income_response.status_code != 200:
        return {"error": "Failed to fetch income statement data"}
    income_data = income_response.json()

    # Fetch Balance Sheet
    balance_response = requests.get(base_url, params=balance_params)
    if balance_response.status_code != 200:
        return {"error": "Failed to fetch balance sheet data"}
    balance_data = balance_response.json()

    # Extract relevant data
    try:
        latest_income = income_data["annualReports"][0]
        latest_balance = balance_data["annualReports"][0]

        return {
            "revenue": float(latest_income["totalRevenue"]),
            "net_income": float(latest_income["netIncome"]),
            "liabilities": float(latest_balance["totalLiabilities"]),
            "assets": float(latest_balance["totalAssets"])
        }
    except (KeyError, IndexError):
        return {"error": "Incomplete financial data"}

# Function to compute financial ratios
def compute_ratios(data):
    """
    Compute financial ratios from the provided data.
    """
    revenue = data.get("revenue", 1)  # Avoid division by zero
    net_income = data.get("net_income", 0)
    liabilities = data.get("liabilities", 0)
    assets = data.get("assets", 1)

    return {
        "Return on Equity (ROE)": net_income / (assets - liabilities) if assets != liabilities else 0,
        "Return on Assets (ROA)": net_income / assets,
        "Gross Margin": (revenue - liabilities) / revenue if revenue else 0,
        "Net Profit Margin": net_income / revenue if revenue else 0
    }

# Define tools for the agent
def fetch_financial_tool(input_text):
    """
    Fetch and calculate financial ratios based on stock ticker input.
    """
    ticker = input_text.upper()
    # api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your Alpha Vantage API key
    data = fetch_financial_data(alpha_vantage_api_key, ticker)
    if "error" in data:
        return data["error"]
    ratios = compute_ratios(data)
    return ratios

tools = [
    Tool(
        name="Fetch Financial Ratios",
        func=fetch_financial_tool,
        description="Fetches financial data and computes ratios based on a stock ticker."
    )
]

# Cohere LLM setup
cohere_llm = Cohere(model="command-xlarge")  # Replace with your Cohere API key

from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a financial assistant. Use the tools provided to answer questions.

Tools available:
{tools}

Names of tools: {tool_names}

Conversation history:
{history}

Current user input:
{input}

Assistant's reasoning and actions:
{agent_scratchpad}
"""
)

# Create the agent
memory = ConversationBufferMemory(memory_key="history")
agent = create_structured_chat_agent(
    llm=cohere_llm,
    tools=tools,
    prompt=prompt,
    output_key="output"
)

# Create executor
executor = AgentExecutor(agent=agent, tools=tools)

ticker = "AAPL"  # Replace with desired stock ticker
print(executor.run(f"Fetch financial ratios for {ticker}"))


TypeError: create_structured_chat_agent() got an unexpected keyword argument 'output_key'

In [50]:
fetch_financial_tool(ticker)

{'Return on Equity (ROE)': 1.6459350307287095,
 'Return on Assets (ROA)': 0.25682503150857583,
 'Gross Margin': 0.21227000140652372,
 'Net Profit Margin': 0.23971255769943867}

In [29]:
main()

Financial Analysis for AAPL:
Ticker: AAPL
Return on Equity (ROE): 164.59%
Return on Assets (ROA): 25.68%
Gross Margin: 46.21%
Net Profit Margin: 23.97%
Debt to Equity Ratio: 5.41
